In [15]:
import mysql.connector  
import pandas as pd
import numpy as np

In [4]:
#Connect to database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="week_9"
)

cursor = connection.cursor()

#Create tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS  Category (
    categoryID INT AUTO_INCREMENT PRIMARY KEY,
    Name VARCHAR(250) NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Product (
    productID INT PRIMARY KEY,
    name VARCHAR(250) NOT NULL,
    categoryID INT,
    price DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (categoryID) REFERENCES Category(categoryID)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Variant (
    variantID INT AUTO_INCREMENT PRIMARY KEY,
    productID INT,
    color VARCHAR(30),
    size VARCHAR(5),
    stock INT,
    FOREIGN KEY (productID) REFERENCES Product(productID)
);
""")

cursor.execute("""
Create table if not exists Customer (   
    customerID INT AUTO_INCREMENT PRIMARY KEY,
    fullName VARCHAR(250) NOT NULL,
    phone VARCHAR(15) NOT NULL,
    address VARCHAR(250) NOT NULL,
    createdDate DATETIME DEFAULT CURRENT_TIMESTAMP
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Orders (
    orderID INT AUTO_INCREMENT PRIMARY KEY,
    customerID INT,
    orderDate DATETIME DEFAULT CURRENT_TIMESTAMP,
    status VARCHAR(20),
    FOREIGN KEY (customerID) REFERENCES Customer(customerID)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Order_Detail (
    orderID INT,
    variantID INT,
    quantity INT,
    price DECIMAL(10, 2),
    FOREIGN KEY (orderID) REFERENCES Orders(orderID),
    FOREIGN KEY (variantID) REFERENCES Variant(variantID)
);
""")

#Commit changes
connection.commit()

#Print result
print("Tables created successfully")

#Close connection
cursor.close()
connection.close()


Tables created successfully


In [6]:
#Connect to database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="week_9"
)

cursor = connection.cursor()

In [7]:
#Insert data into category table
data_category = [("Áo",),("Quần",),("Váy",),("Phụ kiện",)]
cursor.executemany("""
INSERT INTO Category (Name) 
VALUES (%s)
""", data_category)

connection.commit()
print("Category inserted successfully")

Category inserted successfully


In [8]:
#Insert data into product table
data_product = [(101, "Áo sơ mi nam",1, 300000),
                (102, "Quần jean",2, 500000),
                (103, "Váy xòe",3, 450000)]
cursor.executemany("""
INSERT INTO Product (productID,name, categoryID, price)
VALUES (%s, %s, %s, %s)
""", data_product)

connection.commit()
print("Product inserted successfully")

Product inserted successfully


In [9]:
#Insert data into variant table
data_variant = [(101, "Trắng", "M", 20),
                (101, "Trắng", "L", 15),
                (102, "Xanh", "M", 10),
                (103, "Đỏ", "S", 5),]
cursor.executemany("""
INSERT INTO Variant (productID, color, size, stock)
VALUES (%s, %s, %s, %s)
""", data_variant)

connection.commit()
print("Variant inserted successfully")

Variant inserted successfully


In [11]:
#Create customer table
data_customer = [("Trần An", "0909123456", "Hà Nội", "2024-01-10"),
                ("Lê Bình", "0909223456", "TP.HCM", "2024-02-05")]
cursor.executemany("""
INSERT INTO Customer (fullName, phone, address, createdDate)
VALUES (%s, %s, %s, %s)
""", data_customer)

connection.commit()
print("Customer inserted successfully")


Customer inserted successfully


In [12]:
#Insert data into order table
data_order = [(1, "2024-03-01", "Đã giao hàng"),
              (2, "2024-03-02", "Đã đặt hàng")]
cursor.executemany("""
INSERT INTO Orders (customerID, orderDate, status)
VALUES (%s, %s, %s)
""", data_order)

connection.commit()
print("Order inserted successfully")


Order inserted successfully


In [13]:
#Insert data into order detail table
data_order_detail = [(1, 1, 2, 300000),
                     (1, 3, 1, 500000),
                     (2, 4, 1, 450000)]
cursor.executemany("""
INSERT INTO Order_Detail (orderID, variantID, quantity, price)
VALUES (%s, %s, %s, %s)
""", data_order_detail)

connection.commit()
print("Order detail inserted successfully")

Order detail inserted successfully


In [14]:
#Close connection
cursor.close()
connection.close()


In [17]:
#Connect to database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="week_9"
)

cursor = connection.cursor()


In [18]:
#  3. Liệt kê tất cả các đơn hàng, kèm theo tên khách hàng và tổng tiền của mỗi đơn hàng.
query = """
SELECT o.orderID, c.fullName, SUM(od.quantity * od.price) AS totalPrice
FROM Orders o
JOIN Customer c ON o.customerID = c.customerID
JOIN Order_Detail od ON o.orderID = od.orderID
GROUP BY o.orderID, c.fullName
"""
result = pd.read_sql_query(query, connection)
print(result)


   orderID fullName  totalPrice
0        1  Trần An   1100000.0
1        2  Lê Bình    450000.0


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_2560\3896531262.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, connection)


In [20]:
query = """
Select p.Name, v.size, v.color, v.stock
From Product p
Join variant v on v.productID = p.productID
Where v.stock > 10
"""
result = pd.read_sql_query(query, connection)
print(result)


           Name size  color  stock
0  Áo sơ mi nam    M  Trắng     20
1  Áo sơ mi nam    L  Trắng     15


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_2560\301675568.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, connection)


In [21]:
query = """
Select o.orderDate, o.status, Sum(od.quantity * od.price) AS totalAvenue
From Orders o
Join Order_Detail od on o.orderID = od.orderID
Group by o.orderDate, o.status
Order by totalAvenue desc
"""
result = pd.read_sql_query(query, connection)
print(result)



   orderDate        status  totalAvenue
0 2024-03-01  Đã giao hàng    1100000.0
1 2024-03-02   Đã đặt hàng     450000.0


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_2560\794097712.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, connection)


In [29]:
query = """
SELECT c.fullName, o.status, o.orderDate
FROM Customer c
JOIN Orders o ON c.customerID = o.customerID
WHERE o.status IN ("Đã đặt hàng", "Đã giao hàng")
"""
result = pd.read_sql_query(query, connection)
print(result)


  fullName        status  orderDate
0  Trần An  Đã giao hàng 2024-03-01
1  Lê Bình   Đã đặt hàng 2024-03-02


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_2560\897566760.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, connection)


In [30]:
query = """
Select p.categoryID, Sum(od.quantity * od.price) AS totalAvenue
From Product p
Join variant v on v.productID = p.productID
join Order_Detail od on od.variantID = v.variantID
Group by p.categoryID
"""
result = pd.read_sql_query(query, connection)
print(result)



   categoryID  totalAvenue
0           1     600000.0
1           2     500000.0
2           3     450000.0


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_2560\2591013791.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, connection)


In [32]:
query = """
Select p.Name, od.quantity
From Product p
Join variant v on v.productID = p.productID
Join Order_Detail od on od.variantID = v.variantID
Where od.quantity > 1
"""
result = pd.read_sql_query(query, connection)
print(result)



           Name  quantity
0  Áo sơ mi nam         2


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_2560\3240465448.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, connection)


In [33]:
query = """
Select ca.categoryID, ca.Name, Sum(od.quantity * od.price) AS totalAvenue
from Category ca
Join Product p on p.categoryID = ca.categoryID
Join variant v on v.productID = p.productID
Join Order_Detail od on od.variantID = v.variantID
Group by ca.categoryID, ca.Name
"""
result = pd.read_sql_query(query, connection)
print(result)


   categoryID  Name  totalAvenue
0           1    Áo     600000.0
1           2  Quần     500000.0
2           3   Váy     450000.0


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_2560\1039240507.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, connection)


In [37]:
query = """
SELECT 
    ca.Name AS categoryName, 
    p.Name AS productName, 
    v.color, 
    v.size, 
    SUM(od.quantity) AS totalQuantity,
    c.fullName, 
    c.phone, 
    c.address, 
    o.orderDate, 
    SUM(od.quantity * od.price) AS totalAvenue
FROM Category ca
JOIN Product p ON p.categoryID = ca.categoryID
JOIN Variant v ON v.productID = p.productID
JOIN Order_Detail od ON od.variantID = v.variantID
JOIN Orders o ON o.orderID = od.orderID
JOIN Customer c ON c.customerID = o.customerID
GROUP BY 
    ca.Name, 
    p.Name, 
    v.color, 
    v.size, 
    c.fullName, 
    c.phone, 
    c.address, 
    o.orderDate
"""
result = pd.read_sql_query(query, connection)
print(result)





  categoryName   productName  color size  totalQuantity fullName       phone  \
0           Áo  Áo sơ mi nam  Trắng    M            2.0  Trần An  0909123456   
1         Quần     Quần jean   Xanh    M            1.0  Trần An  0909123456   
2          Váy       Váy xòe     Đỏ    S            1.0  Lê Bình  0909223456   

  address  orderDate  totalAvenue  
0  Hà Nội 2024-03-01     600000.0  
1  Hà Nội 2024-03-01     500000.0  
2  TP.HCM 2024-03-02     450000.0  


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_2560\569296799.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, connection)


In [38]:
query = """
Select c.fullName, o.orderID
From Customer c
LEFT JOIN Orders o on c.customerID = o.customerID
Where o.orderID is null
"""
result = pd.read_sql_query(query, connection)
print(result)





Empty DataFrame
Columns: [fullName, orderID]
Index: []


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_2560\2289167059.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, connection)


In [47]:
query = """
Select c.fullName, MONTH(c.createdDate) AS month, YEAR(c.createdDate) AS year
from Customer c
order by month asc
"""
result = pd.read_sql_query(query, connection)
print(result)

  fullName  month  year
0  Trần An      1  2024
1  Lê Bình      2  2024


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_2560\2239870135.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, connection)


In [51]:
query = """
Select o.orderID, o.orderDate,c.fullName, o.status
From Orders o
Join Customer c on c.customerID = o.customerID
order by field(o.status, "Đã đặt hàng", "Đã giao hàng", "Đã hủy hàng")
"""
result = pd.read_sql_query(query, connection)
print(result)


   orderID  orderDate fullName        status
0        2 2024-03-02  Lê Bình   Đã đặt hàng
1        1 2024-03-01  Trần An  Đã giao hàng


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_2560\4185796860.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, connection)


In [53]:
query = """
Select ca.Name,  AVG(v.stock) AS Average_Stock
From Category ca
Join Product p on p.categoryID = ca.categoryID
Join variant v on v.productID = p.productID
Group by ca.Name
"""
result = pd.read_sql_query(query, connection)
print(result)







   Name  Average_Stock
0    Áo           17.5
1  Quần           10.0
2   Váy            5.0


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_2560\2492386805.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, connection)
